In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('train.csv')

In [10]:
df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,136424,M22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,136425,H38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,136426,L54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0
136427,136427,L53876,L,301.7,310.9,1447,46.3,42,0,0,0,0,0,0


In [11]:
df.drop(['id','Product ID','TWF','HDF','PWF','OSF','RNF'],axis = 1, inplace=True)

In [12]:
df

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure
0,L,300.6,309.6,1596,36.1,140,0
1,M,302.6,312.1,1759,29.1,200,0
2,L,299.3,308.5,1805,26.5,25,0
3,L,301.0,310.9,1524,44.3,197,0
4,M,298.0,309.0,1641,35.4,34,0
...,...,...,...,...,...,...,...
136424,M,300.1,311.4,1530,37.5,210,0
136425,H,297.5,308.5,1447,49.1,2,0
136426,L,300.5,311.8,1524,38.5,214,0
136427,L,301.7,310.9,1447,46.3,42,0


In [13]:
cols = list(df.columns)

In [14]:
cols

['Type',
 'Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Machine failure']

In [47]:
df["Machine failure"].value_counts()

0    134281
1      2148
Name: Machine failure, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [18]:
ct = ColumnTransformer(transformers=[('tnf1',OrdinalEncoder(),['Type']),('tnf2',StandardScaler(),cols[1:6])],remainder='passthrough')

In [19]:
train_data = ct.fit_transform(df)

In [20]:
X = train_data[:,:-1]
y = train_data[:,-1]

In [21]:
train_X,test_X,train_y,test_y = train_test_split(X,y,stratify=y,test_size=0.3)

In [37]:
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score

## K Means

In [33]:
from sklearn.cluster import KMeans

In [34]:
km = KMeans(n_clusters=2)

In [35]:
km.fit(train_X,train_y)

KMeans(n_clusters=2)

In [38]:
y_pred = km.predict(test_X)

print("Accuracy:",accuracy_score(y_pred,test_y))
print("Precision Score",precision_score(y_pred,test_y))
print("Recall",recall_score(y_pred,test_y))
print("f1_Score",f1_score(y_pred,test_y))

Accuracy: 0.5047032666324611
Precision Score 0.2841614906832298
Recall 0.009152745823747125
f1_Score 0.01773427657718771


## Agglomerative Clustering

In [43]:
from sklearn.cluster import AgglomerativeClustering

In [44]:
agg_cluster = AgglomerativeClustering(n_clusters=2, linkage='ward')

In [45]:
y_aggo = agg_cluster.fit_predict(np.array(train_X[0:2000],dtype="float16"))

In [46]:
print("Accuracy:",accuracy_score(y_aggo,train_y[0:2000]))
print("Precision Score",precision_score(y_aggo,train_y[0:2000]))
print("Recall",recall_score(y_aggo,train_y[0:2000]))
print("f1_Score",f1_score(y_aggo,train_y[0:2000]))

Accuracy: 0.5355
Precision Score 0.65625
Recall 0.022364217252396165
f1_Score 0.04325437693099897


## PCA and random forest